In [3]:
from keras.callbacks import CSVLogger, ModelCheckpoint, EarlyStopping
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.layers import Activation, Convolution2D, Dropout, Conv2D
from keras.layers import AveragePooling2D, BatchNormalization
from keras.layers import GlobalAveragePooling2D
from keras.models import Sequential
from keras.layers import Flatten
from keras.models import Model
from keras.layers import Input
from keras.layers import MaxPooling2D
from keras.layers import SeparableConv2D
from keras import layers
from keras.regularizers import l2
import pandas as pd
import cv2
import numpy as np

Using TensorFlow backend.


In [4]:
dataset_path = "C://Users//shifa//Desktop//facial//fer2013.csv"
image_size=(48,48)
 
def load_fer2013():
    data = pd.read_csv(dataset_path)
    pixels = data['pixels'].tolist()
    width, height = 48, 48
    faces = []
    for pixel_sequence in pixels:
        face = [int(pixel) for pixel in pixel_sequence.split(' ')]
        face = np.asarray(face).reshape(width, height)
        face = cv2.resize(face.astype('uint8'),image_size)
        faces.append(face.astype('float32'))
    faces = np.asarray(faces)
    faces = np.expand_dims(faces, -1)
    emotions = pd.get_dummies(data['emotion']).as_matrix()
    return faces, emotions
 
def preprocess_input(x, v2=True):
    x = x.astype('float32')
    x = x / 255.0
    if v2:
        x = x - 0.5
        x = x * 2.0
    return x
 
faces, emotions = load_fer2013()
faces = preprocess_input(faces)
xtrain, xtest,ytrain,ytest = train_test_split(faces, emotions,test_size=0.2,shuffle=True)

c:\users\shifa\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  app.launch_new_instance()


In [6]:
# parameters
batch_size = 32
num_epochs = 110
input_shape = (48, 48, 1)
verbose = 1
num_classes = 7
patience = 50
base_path = 'C://Users//shifa//Desktop//facial//'
l2_regularization=0.01
 
# data generator
data_generator = ImageDataGenerator(
                        featurewise_center=False,
                        featurewise_std_normalization=False,
                        rotation_range=10,
                        width_shift_range=0.1,
                        height_shift_range=0.1,
                        zoom_range=.1,
                        horizontal_flip=True)
# model parameters
regularization = l2(l2_regularization)
 
# base
img_input = Input(input_shape)
x = Conv2D(8, (3, 3), strides=(1, 1), kernel_regularizer=regularization, use_bias=False)(img_input)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Conv2D(8, (3, 3), strides=(1, 1), kernel_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

# module 1
residual = Conv2D(16, (1, 1), strides=(2, 2), padding='same', use_bias=False)(x)
residual = BatchNormalization()(residual)
x = SeparableConv2D(16, (3, 3), padding='same', kernel_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = SeparableConv2D(16, (3, 3), padding='same', kernel_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
x = layers.add([x, residual])
 
# module 2
residual = Conv2D(32, (1, 1), strides=(2, 2), padding='same', use_bias=False)(x)
residual = BatchNormalization()(residual)
x = SeparableConv2D(32, (3, 3), padding='same', kernel_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = SeparableConv2D(32, (3, 3), padding='same', kernel_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
x = layers.add([x, residual])
 
# module 3
residual = Conv2D(64, (1, 1), strides=(2, 2),padding='same', use_bias=False)(x)
residual = BatchNormalization()(residual)
x = SeparableConv2D(64, (3, 3), padding='same',kernel_regularizer=regularization,use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = SeparableConv2D(64, (3, 3), padding='same',kernel_regularizer=regularization,use_bias=False)(x)
x = BatchNormalization()(x)
x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
x = layers.add([x, residual])

# module 4
residual = Conv2D(128, (1, 1), strides=(2, 2),padding='same', use_bias=False)(x)
residual = BatchNormalization()(residual)
x = SeparableConv2D(128, (3, 3), padding='same',kernel_regularizer=regularization,use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = SeparableConv2D(128, (3, 3), padding='same',kernel_regularizer=regularization,use_bias=False)(x)
x = BatchNormalization()(x)
x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
x = layers.add([x, residual])
x = Conv2D(num_classes, (3, 3), padding='same')(x)
x = GlobalAveragePooling2D()(x)
output = Activation('softmax',name='predictions')(x)
 
model = Model(img_input, output)
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()
 
# callbacks
log_file_path = base_path + '_emotion_training.log'
csv_logger = CSVLogger(log_file_path, append=False)
early_stop = EarlyStopping('val_loss', patience=patience)
reduce_lr = ReduceLROnPlateau('val_loss', factor=0.1, patience=int(patience/4), verbose=1)
trained_models_path = base_path + '_mini_XCEPTION'
model_names = trained_models_path + '.{epoch:02d}-{val_acc:.2f}.hdf5'
model_checkpoint = ModelCheckpoint(model_names, 'val_loss', verbose=1,save_best_only=True)
callbacks = [model_checkpoint, csv_logger, early_stop, reduce_lr]
 
model.fit_generator(data_generator.flow(xtrain, ytrain,batch_size),
                        steps_per_epoch=len(xtrain) / batch_size,
                        epochs=num_epochs, verbose=1, callbacks=callbacks,
                        validation_data=(xtest,ytest))

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 48, 48, 1)    0                                            
__________________________________________________________________________________________________
conv2d_8 (Conv2D)               (None, 46, 46, 8)    72          input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_15 (BatchNo (None, 46, 46, 8)    32          conv2d_8[0][0]                   
__________________________________________________________________________________________________
activation_7 (Activation)       (None, 46, 46, 8)    0           batch_normalization_15[0][0]     
__________________________________________________________________________________________________
conv2d_9 (

Epoch 1/110
898/897 [==============================] - 117s 130ms/step - loss: 1.7787 - acc: 0.3268 - val_loss: 1.6401 - val_acc: 0.3936

Epoch 00001: val_loss improved from inf to 1.64007, saving model to C://Users//shifa//Desktop//facial//_mini_XCEPTION.01-0.39.hdf5
Epoch 2/110
898/897 [==============================] - 102s 113ms/step - loss: 1.5285 - acc: 0.4323 - val_loss: 1.6356 - val_acc: 0.4089

Epoch 00002: val_loss improved from 1.64007 to 1.63557, saving model to C://Users//shifa//Desktop//facial//_mini_XCEPTION.02-0.41.hdf5
Epoch 3/110
898/897 [==============================] - 102s 113ms/step - loss: 1.4218 - acc: 0.4710 - val_loss: 1.3733 - val_acc: 0.4836

Epoch 00003: val_loss improved from 1.63557 to 1.37334, saving model to C://Users//shifa//Desktop//facial//_mini_XCEPTION.03-0.48.hdf5
Epoch 4/110
898/897 [==============================] - 102s 113ms/step - loss: 1.3392 - acc: 0.5002 - val_loss: 1.2799 - val_acc: 0.5208

Epoch 00004: val_loss improved from 1.37334 to 


Epoch 00037: val_loss improved from 1.04257 to 1.02870, saving model to C://Users//shifa//Desktop//facial//_mini_XCEPTION.37-0.62.hdf5
Epoch 38/110
898/897 [==============================] - 111s 123ms/step - loss: 0.9887 - acc: 0.6297 - val_loss: 1.0892 - val_acc: 0.5979

Epoch 00038: val_loss did not improve from 1.02870
Epoch 39/110
898/897 [==============================] - 104s 115ms/step - loss: 0.9826 - acc: 0.6322 - val_loss: 1.0233 - val_acc: 0.6202

Epoch 00039: val_loss improved from 1.02870 to 1.02334, saving model to C://Users//shifa//Desktop//facial//_mini_XCEPTION.39-0.62.hdf5
Epoch 40/110
898/897 [==============================] - 101s 113ms/step - loss: 0.9769 - acc: 0.6343 - val_loss: 1.0277 - val_acc: 0.6135

Epoch 00040: val_loss did not improve from 1.02334
Epoch 41/110
898/897 [==============================] - 101s 113ms/step - loss: 0.9716 - acc: 0.6393 - val_loss: 1.0296 - val_acc: 0.6177

Epoch 00041: val_loss did not improve from 1.02334
Epoch 42/110
898/897

898/897 [==============================] - 106s 118ms/step - loss: 0.8406 - acc: 0.6903 - val_loss: 0.9548 - val_acc: 0.6486

Epoch 00076: val_loss improved from 0.95728 to 0.95479, saving model to C://Users//shifa//Desktop//facial//_mini_XCEPTION.76-0.65.hdf5
Epoch 77/110
898/897 [==============================] - 104s 116ms/step - loss: 0.8340 - acc: 0.6911 - val_loss: 0.9526 - val_acc: 0.6493

Epoch 00077: val_loss improved from 0.95479 to 0.95259, saving model to C://Users//shifa//Desktop//facial//_mini_XCEPTION.77-0.65.hdf5
Epoch 78/110
898/897 [==============================] - 112s 125ms/step - loss: 0.8374 - acc: 0.6923 - val_loss: 0.9605 - val_acc: 0.6456

Epoch 00078: val_loss did not improve from 0.95259
Epoch 79/110
898/897 [==============================] - 111s 124ms/step - loss: 0.8290 - acc: 0.6917 - val_loss: 0.9619 - val_acc: 0.6411

Epoch 00079: val_loss did not improve from 0.95259
Epoch 80/110
898/897 [==============================] - 113s 126ms/step - loss: 0.831